<a href="https://colab.research.google.com/github/Ebasurtos/AHPC/blob/main/An%C3%A1lisis_de_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# performance_analysis_corrected.py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
from scipy import stats

class ParallelPerformanceModel:
    """Modelo teórico de performance para la ecuación de calor 2D paralela"""

    def __init__(self, N_total, p, alpha=1e-6, beta=1e-9):
        """
        N_total: puntos totales (N × N)
        p: número de procesos
        alpha: tiempo por operación de punto flotante (segundos)
        beta: tiempo por byte de comunicación (segundos/byte)
        """
        self.N = int(np.sqrt(N_total))
        self.p = p
        self.alpha = alpha
        self.beta = beta

    def compute_serial_time(self):
        """Tiempo secuencial teórico"""
        # Operaciones por celda por iteración: ~10 flops
        flops_per_cell = 10
        total_flops = self.N * self.N * flops_per_cell
        return total_flops * self.alpha

    def compute_parallel_time(self, iters=100):
        """Tiempo paralelo teórico por iteración"""
        N_local = self.N / np.sqrt(self.p)

        # Tiempo de cómputo
        flops_per_cell = 10
        comp_time = N_local * N_local * flops_per_cell * self.alpha

        # Tiempo de comunicación (halos en 4 direcciones)
        halo_size = N_local * 8  # 8 bytes por double
        comm_time = 4 * (self.beta * halo_size)  # 4 vecinos

        # Tiempo de sincronización (Allreduce)
        sync_time = np.log2(self.p) * 100 * self.alpha

        return (comp_time + comm_time + sync_time) * iters

    def compute_speedup(self):
        """Speedup teórico"""
        T_serial = self.compute_serial_time()
        T_parallel = self.compute_parallel_time()
        return T_serial / T_parallel

    def compute_efficiency(self):
        """Eficiencia teórica"""
        return self.compute_speedup() / self.p

    def compute_scalability_limit(self):
        """Límite de escalabilidad (Gustafson's Law)"""
        # Fórmula: S(p) = p + (1-p)*s, donde s es fracción secuencial
        s = 0.01  # estimado del 1% secuencial
        return self.p + (1 - self.p) * s

def analyze_communication_pattern():
    """Analiza patrones de comunicación para diferentes topologías"""

    topologies = ['1D Pipeline', '2D Mesh', '2D Torus', 'Hypercube']
    p_values = [2**i for i in range(1, 7)]  # 2, 4, 8, 16, 32, 64

    results = []

    for p in p_values:
        # Comunicación para topología 2D Mesh (asumida en el código)
        mesh_dim = int(np.sqrt(p))
        if mesh_dim**2 == p:  # Cuadrado perfecto
            # Cada proceso tiene 4 vecinos (excepto bordes)
            avg_neighbors = 4 * (1 - 2/mesh_dim + 1/(mesh_dim**2))
            comm_volume = 2 * avg_neighbors * 8  # 2 direcciones, 8 bytes

            results.append({
                'Processes': p,
                'Topology': '2D Mesh',
                'Diameter': 2*(mesh_dim-1),
                'AvgDegree': avg_neighbors,
                'BisectionWidth': mesh_dim,
                'CommVolume': comm_volume
            })

    return pd.DataFrame(results)

def plot_scalability_analysis():
    """Genera análisis completo de escalabilidad"""

    fig = plt.figure(figsize=(16, 10))

    # 1. Strong Scaling
    ax1 = plt.subplot(2, 3, 1)
    N = 10000
    p_values = np.arange(1, 65)

    speedups_ideal = p_values
    speedups_real = []

    for p in p_values:
        model = ParallelPerformanceModel(N, p)
        speedups_real.append(model.compute_speedup())

    ax1.plot(p_values, speedups_ideal, 'r--', label='Ideal', linewidth=2)
    ax1.plot(p_values, speedups_real, 'b-', label='Real', linewidth=2)
    ax1.fill_between(p_values, speedups_real, speedups_ideal, alpha=0.2, color='red')
    ax1.set_xlabel('Number of Processes')
    ax1.set_ylabel('Speedup')
    ax1.set_title('Strong Scaling Analysis')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Weak Scaling
    ax2 = plt.subplot(2, 3, 2)
    p_values = [1, 4, 16, 64]
    problem_sizes = [10000, 40000, 160000, 640000]  # Aumenta con p

    efficiencies = []
    for p, N in zip(p_values, problem_sizes):
        model = ParallelPerformanceModel(N, p)
        efficiencies.append(model.compute_efficiency())

    ax2.bar(range(len(p_values)), efficiencies, color=['blue', 'green', 'orange', 'red'])
    ax2.set_xticks(range(len(p_values)))
    ax2.set_xticklabels([f'p={p}\nN={int(np.sqrt(N))}' for p, N in zip(p_values, problem_sizes)])
    ax2.set_ylabel('Efficiency')
    ax2.set_title('Weak Scaling Analysis')
    ax2.axhline(y=1.0, color='r', linestyle='--', alpha=0.5)
    ax2.grid(True, alpha=0.3, axis='y')

    # 3. Kiviat Diagram for Performance Metrics
    ax3 = plt.subplot(2, 3, 3, projection='polar')

    metrics = ['Speedup', 'Efficiency', 'Scalability', 'Load Balance', 'Comm/Comp Ratio']
    values = [0.7, 0.65, 0.8, 0.9, 0.3]  # Valores de ejemplo normalizados

    angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False).tolist()
    values += values[:1]
    angles += angles[:1]

    ax3.plot(angles, values, 'o-', linewidth=2)
    ax3.fill(angles, values, alpha=0.25)
    ax3.set_xticks(angles[:-1])
    ax3.set_xticklabels(metrics)
    ax3.set_title('Performance Metrics Radar Chart', pad=20)

    # 4. Communication/Computation Ratio
    ax4 = plt.subplot(2, 3, 4)

    p_values = np.logspace(0, 2, 20)
    ratios = []

    for p in p_values:
        model = ParallelPerformanceModel(10000, int(p))
        comp = model.compute_serial_time() / p
        # Estimación simplificada de comunicación
        N_local = 100 / np.sqrt(p)
        comm = 4 * N_local * 8 * model.beta
        ratios.append(comm / comp if comp > 0 else 0)

    ax4.semilogx(p_values, ratios, 'g-', linewidth=2)
    ax4.axhline(y=1.0, color='r', linestyle='--', label='Comm = Comp')
    ax4.set_xlabel('Number of Processes (log scale)')
    ax4.set_ylabel('Communication / Computation Ratio')
    ax4.set_title('Comm/Comp Ratio vs Processes')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # 5. Iso-efficiency Analysis
    ax5 = plt.subplot(2, 3, 5)

    p = np.arange(1, 65)
    # W = K * p * Tp, donde W es trabajo, Tp es tiempo paralelo
    iso_efficiency = p / (1 + 0.01 * np.sqrt(p))  # Ejemplo simplificado

    ax5.plot(p, iso_efficiency, 'purple', linewidth=2)
    ax5.set_xlabel('Number of Processes')
    ax5.set_ylabel('Problem Size (Normalized)')
    ax5.set_title('Iso-efficiency Function')
    ax5.grid(True, alpha=0.3)

    # 6. Performance Model Comparison
    ax6 = plt.subplot(2, 3, 6)

    p = np.arange(1, 65)

    # Diferentes modelos
    amdahl = 1 / (0.05 + 0.95/p)  # 5% serial
    gustafson = 0.05 + 0.95*p     # 5% serial
    real = p / (1 + 0.1*np.log2(p) + 0.01*np.sqrt(p))  # Modelo realista

    ax6.plot(p, p, 'k--', label='Ideal', alpha=0.5)
    ax6.plot(p, amdahl, 'r-', label="Amdahl's Law", linewidth=2)
    ax6.plot(p, gustafson, 'g-', label="Gustafson's Law", linewidth=2)
    ax6.plot(p, real, 'b-', label='Realistic Model', linewidth=2)
    ax6.set_xlabel('Number of Processes')
    ax6.set_ylabel('Speedup')
    ax6.set_title('Performance Models Comparison')
    ax6.legend()
    ax6.grid(True, alpha=0.3)

    plt.suptitle('Complete Parallel Performance Analysis for Heat Equation', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.savefig('performance_analysis_complete.png', dpi=300, bbox_inches='tight')
    plt.show()

def generate_performance_report():
    """Genera reporte detallado de performance"""

    report = """
    ========================================================
    PERFORMANCE ANALYSIS REPORT: 2D HEAT EQUATION SOLVER
    ========================================================

    1. ALGORITHM OVERVIEW
    ---------------------
    Algorithm: Jacobi iteration for 2D heat equation
    Parallel Model: SPMD with domain decomposition
    Communication: MPI point-to-point (halo exchange) + collective operations

    2. SEQUENCE OF OPERATIONS (per iteration):
    ------------------------------------------
    Step 1: Halo Communication (non-blocking)
            - MPI_Isend/MPI_Irecv for 4 directions
            - MPI_Waitall for synchronization

    Step 2: Local Computation
            - Finite difference calculation
            - Local max difference computation

    Step 3: Global Synchronization (every 'stride' iterations)
            - MPI_Allreduce for global max
            - Convergence check

    Step 4: Update local values

    3. COMPLEXITY ANALYSIS:
    -----------------------
    a) Computation per iteration: O(N²/p) flops
    b) Communication per iteration: O(√(N²/p)) bytes
    c) Synchronization: O(log p) every 'stride' iterations

    4. SCALABILITY ASSESSMENT:
    --------------------------
    Strong Scaling Limit: Limited by communication overhead
    Weak Scaling Efficiency: Good for balanced domain decomposition
    Bottlenecks: Halo communication and global reductions

    5. OPTIMIZATION RECOMMENDATIONS:
    --------------------------------
    1. Use MPI_Neighbor_alltoall for structured grids
    2. Implement communication-computation overlap
    3. Adjust 'stride' parameter based on problem size
    4. Consider asynchronous iteration methods

    6. EXPECTED PERFORMANCE METRICS:
    ---------------------------------
    Metric                  | Good       | Fair       | Poor
    -------------------------------------------------------
    Parallel Efficiency     | > 70%      | 40-70%     | < 40%
    Comm/Comp Ratio        | < 0.1      | 0.1-0.3    | > 0.3
    Load Imbalance         | < 5%       | 5-15%      | > 15%

    ========================================================
    """

    with open('performance_report.txt', 'w') as f:
        f.write(report)

    print(report)

# Ejecutar análisis completo
if __name__ == "__main__":
    print("Iniciando análisis de performance...")

    # 1. Análisis de patrones de comunicación
    df_comm = analyze_communication_pattern()
    print("Análisis de comunicación:")
    print(df_comm.to_string())

    # 2. Generar gráficos de escalabilidad
    plot_scalability_analysis()

    # 3. Generar reporte
    generate_performance_report()

    # 4. Modelo de performance para diferentes configuraciones
    print("\nModelo de performance para diferentes configuraciones:")
    configs = [
        (10000, 1),
        (10000, 4),
        (10000, 16),
        (10000, 64)
    ]

    for N, p in configs:
        model = ParallelPerformanceModel(N, p)
        print(f"\nN={N}, p={p}:")
        print(f"  Speedup teórico: {model.compute_speedup():.2f}")
        print(f"  Eficiencia teórica: {model.compute_efficiency():.2%}")

    print("\nAnálisis completado. Archivos generados:")
    print("1. performance_analysis_complete.png")
    print("2. performance_report.txt")
    print("3. communication_analysis.csv")